In [33]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL,LambdaLayer,biasLayer
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])
# without tanh is worse-off

In [126]:
## change ##
batch_size = 16
clip = .25
block = ResidualBlockGLU

Prepare data

In [4]:
train_gen = SequenceGen(train)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [131]:
model = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      block(32),
                      Dropout(0.5),
                      block(64),
                      Dropout(0.5),
                      block(128),
                      Dropout(0.5),
#                       DenseBlock(256),
#                       Dropout(0.5),
#                       DenseBlock(512),
#                       Dropout(0.5),                      
                      LambdaLayer(lambda x:x.transpose(1,2)),
                      Linear(256,1,bias=False),
                      LeakyReLU(0.2,True),
                      LambdaLayer(lambda x:x.squeeze(2).mean(1,keepdim=True)),
                      #LambdaLayer(lambda x:x.squeeze(2).median(1,keepdim=True)[0]),                   
                      biasLayer(1),
                      #Linear(1,1),
                      LambdaLayer(lambda x:x.squeeze(1))
                      ).to('cuda:0')

In [132]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [133]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [ ]:
# ResidualBlockGLU >> 2.32
# ResidualBlock >> 2.39

In [134]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:4.966113268182828, val_loss:4.784550666809082
epoch:1, train_loss:4.081194596794935, val_loss:3.867095708847046
epoch:2, train_loss:3.151384955415359, val_loss:2.3459882736206055
epoch:3, train_loss:2.235590308904648, val_loss:2.3524112701416016
epoch:4, train_loss:2.1221670112930813, val_loss:2.4041547775268555
Training completed in 43.60692095756531s


Submission

In [135]:
submission = pd.read_csv('../Data/sample_submission.csv',)
test_gen = SequenceGenTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)
submission.iloc[:,1] = predict(model,test_gen)
submission.to_csv('../Submission/sub_raw_shareWeight.csv',index=False)